In [ ]:
import numpy as np
import pylab as p
import matplotlib.pyplot as plt
import math
import pandas as pd
from google.colab import drive
import os
import json
from random import *
from sympy import *
import torch
import jax
import jax.numpy as jnp
from jax import grad, config
jax.config.update("jax_debug_nans", True)
from copy import deepcopy

In [ ]:
path_to_sepsis_with_lab_values_csv = "/content/data.csv"

sepsis_with_lab_values_df = pd.read_csv(path_to_sepsis_with_lab_values_csv)

print(sepsis_with_lab_values_df.shape)

sepsis_with_lab_values_df.describe()

sepsis_with_lab_values_df.head()

count  = 0

list_of_patient_values = []
list_of_rows = []
output_rows = []

output_rows.append(sepsis_with_lab_values_df.columns.values)

for _, row in sepsis_with_lab_values_df.iterrows():
  count = count + 1
  patient_values = {}
  patient_values["respiration"] = ([],[],[])
  patient_values["coagulation"] = ([],[])
  patient_values["liver"] = ([],[])
  patient_values["cardiovascular"] = ([],[],[],[],[],[])
  patient_values["cns"] = ([],[])
  patient_values["renal"] = ([],[],[])
  patient_values["sofa_score"] = None
  patient_values["respiration"][0].append(row["pao2fio2ratio_novent"])
  patient_values["respiration"][1].append(row["pao2fio2ratio_vent"])
  patient_values["respiration"][2].append(row["respiration"])
  patient_values["coagulation"][0].append(row["platelet_min"])
  patient_values["coagulation"][1].append(row["coagulation"])
  patient_values["liver"][0].append(row["bilirubin_max"])
  patient_values["liver"][1].append(row["liver"])
  patient_values["cardiovascular"][0].append(row["rate_dopamine"])
  patient_values["cardiovascular"][1].append(row["rate_epinephrine"])
  patient_values["cardiovascular"][2].append(row["rate_norepinephrine"])
  patient_values["cardiovascular"][3].append(row["rate_dobutamine"])
  patient_values["cardiovascular"][4].append(row["meanbp_min"])
  patient_values["cardiovascular"][5].append(row["cardiovascular"])
  patient_values["cns"][0].append(row["gcs_min"])
  patient_values["cns"][1].append(row["cns"])
  patient_values["renal"][0].append(row["creatinine_max"])
  patient_values["renal"][1].append(row["uo_24hr"])
  patient_values["renal"][2].append(row["renal"])
  patient_values["sofa_score"] = row["sofa_score"]
  patient_values["sepsis"] = 1
  list_of_patient_values.append(patient_values)
  list_of_rows.append(row)

print(count, " values loaded!")

sepsis_with_lab_values_df.hist(column=["pao2fio2ratio_vent","respiration"])
sepsis_with_lab_values_df.hist(column=["pao2fio2ratio_novent","respiration"])
sepsis_with_lab_values_df.hist(column=["platelet_min","coagulation"])
sepsis_with_lab_values_df.hist(column=["bilirubin_max","liver"])
sepsis_with_lab_values_df.hist(column=["rate_dopamine","cardiovascular"])
sepsis_with_lab_values_df.hist(column=["rate_epinephrine","cardiovascular"])
sepsis_with_lab_values_df.hist(column=["rate_norepinephrine","cardiovascular"])
sepsis_with_lab_values_df.hist(column=["rate_dobutamine","cardiovascular"])
sepsis_with_lab_values_df.hist(column=["meanbp_min","cardiovascular"])
sepsis_with_lab_values_df.hist(column=["gcs_min","cns"])
sepsis_with_lab_values_df.hist(column=["creatinine_max","renal"])
sepsis_with_lab_values_df.hist(column=["uo_24hr","renal"])
sepsis_with_lab_values_df.hist(column="death_flag")

def h_respiration_discrete_original(pao2_fio2_ratio_novent, pao2_fio2_ratio_vent):
    if pao2_fio2_ratio_novent is None and pao2_fio2_ratio_vent is None:
        respiration_sofa is None
    elif pao2_fio2_ratio_vent < 100:
        respiration_sofa = 4
    elif pao2_fio2_ratio_vent < 200:
        respiration_sofa = 3
    elif pao2_fio2_ratio_novent < 300:
        respiration_sofa = 2
    elif pao2_fio2_ratio_vent < 300:
        respiration_sofa = 2
    elif pao2_fio2_ratio_novent < 400:
        respiration_sofa = 1
    elif pao2_fio2_ratio_vent < 400:
        respiration_sofa = 1
    else:
        respiration_sofa = 0
    return respiration_sofa


def h_respiration_vent_cont(pao2_fio2_ratio_vent, beta=10):
  respiration_sofa_vent_cont = 1/(1+jnp.exp(beta*((pao2_fio2_ratio_vent/100)-1))) + 1/(1+jnp.exp(beta*((pao2_fio2_ratio_vent/100)-2))) +1/(1+jnp.exp(beta*((pao2_fio2_ratio_vent/100)-3))) + 1/(1+jnp.exp(beta*((pao2_fio2_ratio_vent/100)-4)))
  return respiration_sofa_vent_cont


def h_respiration_novent_cont(pao2_fio2_ratio_novent, beta=10):
  respiration_sofa_novent_cont = 1/(1+jnp.exp(beta*((pao2_fio2_ratio_novent/100)-3))) + 1/(1+jnp.exp(beta*((pao2_fio2_ratio_novent/100)- 4)))
  return respiration_sofa_novent_cont


def h_respiration_cont(pao2_fio2_ratio_novent, pao2_fio2_ratio_vent, alpha=5):
  respiration_sofa_vent_cont = h_respiration_vent_cont(pao2_fio2_ratio_vent)
  respiration_sofa_novent_cont = h_respiration_novent_cont(pao2_fio2_ratio_novent)
  respiration_sofa_cont = (respiration_sofa_vent_cont*jnp.exp(alpha*respiration_sofa_vent_cont) + respiration_sofa_novent_cont*jnp.exp(alpha*respiration_sofa_novent_cont))/(jnp.exp(alpha*respiration_sofa_vent_cont) + jnp.exp(alpha*respiration_sofa_novent_cont))
  return respiration_sofa_cont


def h_renal_discrete_original(creatinine_max, uo_24hr):
  if creatinine_max >= 5:
    renal_sofa = 4
  elif uo_24hr < 200:
    renal_sofa = 4
  elif creatinine_max >= 3.5:
    renal_sofa = 3
  elif uo_24hr < 500:
    renal_sofa = 3
  elif creatinine_max >= 2:
    renal_sofa = 2
  elif creatinine_max >= 1.2:
    renal_sofa = 1
  elif creatinine_max is None and uo_24hr is None:
    renal_sofa = None
  else:
    renal_sofa = 0
  return renal_sofa


def h_renal_cont_creatinine(creatinine_max, beta=2):
  renal_sofa_cont_creatinine = 1/(1+jnp.exp(beta*((-1)*(creatinine_max - 1.2)))) + 1/(1+jnp.exp(beta*((-1)*(creatinine_max - 2)))) + 1/(1+jnp.exp(beta*((-1)*(creatinine_max - 3.5)))) + 1/(1+jnp.exp(beta*((-1)*(creatinine_max - 5)))) 
  return renal_sofa_cont_creatinine


def h_renal_cont_uo_24hr(uo_24hr,beta=2):
  renal_sofa_cont_uo_24hr = 1/(1+jnp.exp(beta*(((uo_24hr/100) - 2)))) + 3/(1+jnp.exp(beta*(((uo_24hr/100) - 5)))) 
  return renal_sofa_cont_uo_24hr
  

def h_renal_cont(creatinine_max, uo_24hr, alpha=5):
  creatinine_max_op = h_renal_cont_creatinine(creatinine_max)
  uo_24_hr_op = h_renal_cont_uo_24hr(uo_24hr)
  renal_sofa = (creatinine_max_op*jnp.exp(alpha*creatinine_max_op) + uo_24_hr_op*jnp.exp(alpha*uo_24_hr_op))/(jnp.exp(alpha*creatinine_max_op) + jnp.exp(alpha*uo_24_hr_op))
  return renal_sofa


def h_cardiovascular_discrete_original(rate_dopamine, rate_epinephrine, rate_norepinephrine, rate_dobutamine, meanbp_min):
  if rate_dopamine is None and rate_epinephrine is None and rate_norepinephrine is None and rate_dobutamine is None and rate_dopamine is None and meanbp_min is None:
    cardiovascular_sofa = None
  else:
    if rate_dopamine > 15:
      cardiovascular_sofa = 4
    elif rate_epinephrine > 0.1:
      cardiovascular_sofa = 4
    elif rate_norepinephrine > 0.1:
      cardiovascular_sofa = 4
    elif rate_dopamine > 5:
      cardiovascular_sofa = 3
    elif rate_epinephrine > 0:
      cardiovascular_sofa = 3
    elif rate_norepinephrine > 0:
      cardiovascular_sofa = 3
    elif rate_dopamine > 0:
      cardiovascular_sofa = 2
    elif rate_dobutamine > 0:
      cardiovascular_sofa = 2
    elif meanbp_min < 70:
      cardiovascular_sofa = 1
    else:
      cardiovascular_sofa = 0
  return cardiovascular_sofa


def h_cardiovascular_dopamine_cont(rate_dopamine, beta=1):
  cardiovascular_dopamine_cont_sofa = 1/(1+jnp.exp(beta*((-1)*(rate_dopamine/10 - 1.5)))) + 1/(1+jnp.exp(beta*((-1)*(rate_dopamine/10 - 5)))) + 2/(1+jnp.exp(beta*((-1)*(rate_dopamine/10))))
  return cardiovascular_dopamine_cont_sofa


def h_cardiovascular_epinephrine_cont(rate_epinephrine, beta=1):
  cardiovascular_epinephrine_cont_sofa = 1/(1+jnp.exp(beta*((-1)*((rate_epinephrine) - 0.1)))) + 3/(1+jnp.exp(beta*((-1)*((rate_epinephrine)))))
  return cardiovascular_epinephrine_cont_sofa


def h_cardiovascular_norepinephrine_cont(rate_norepinephrine, beta=1):
  cardiovascular_norepinephrine_cont_sofa = 1/(1+jnp.exp(beta*((-1)*((rate_norepinephrine) - 0.1)))) + 3/(1+jnp.exp(beta*((-1)*((rate_norepinephrine)))))
  return cardiovascular_norepinephrine_cont_sofa


def h_cardiovascular_dobutamine_cont(rate_dobutamine, beta=1):
  cardiovascular_dobutamine_cont = 2/(1+jnp.exp(beta*((-1)*(rate_dobutamine))))
  return cardiovascular_dobutamine_cont

def h_cardiovascular_meanbp_min_cont(meanbp_min, beta=1):
  cardiovascular_meanbp_min_cont = 1/(1+jnp.exp(beta*((meanbp_min/100) - 0.7)))
  return cardiovascular_meanbp_min_cont


def h_cardiovascular_cont(rate_dopamine, rate_epinephrine, rate_norepinephrine, rate_dobutamine, meanbp_min, alpha=5):
  dopamine_score = h_cardiovascular_dopamine_cont(rate_dopamine)
  epinephrine_score = h_cardiovascular_epinephrine_cont(rate_epinephrine)
  norepinephrine_score = h_cardiovascular_norepinephrine_cont(rate_norepinephrine)
  dobutamine_score = h_cardiovascular_dobutamine_cont(rate_dobutamine)
  meanbp_min_score = h_cardiovascular_meanbp_min_cont(meanbp_min)
  cardiovascular_sofa_cont = (dopamine_score*jnp.exp(alpha*dopamine_score) + epinephrine_score*jnp.exp(alpha*epinephrine_score) + norepinephrine_score*jnp.exp(alpha*norepinephrine_score) + dobutamine_score*jnp.exp(alpha*dobutamine_score) + meanbp_min_score*jnp.exp(alpha*meanbp_min_score))/(jnp.exp(alpha*dopamine_score) + jnp.exp(alpha*dobutamine_score) + jnp.exp(alpha*epinephrine_score) + jnp.exp(alpha*norepinephrine_score) + jnp.exp(alpha*meanbp_min_score))
  return cardiovascular_sofa_cont

def h_cns_discrete_original(gcs_min):
  if gcs_min is None:
    cns_sofa = None
  elif gcs_min < 6:
    cns_sofa = 4
  elif gcs_min <= 9:
    cns_sofa = 3
  elif gcs_min <= 12:
    cns_sofa = 2
  elif gcs_min < 14:
    cns_sofa = 1
  else:
    cns_sofa = 0
  return cns_sofa

def h_cns_cont(gcs_min, beta=2):
  cns_sofa_cont = 1/(1+jnp.exp(beta*(gcs_min - 6))) + 1/(1+jnp.exp(beta*(gcs_min - 9))) + 1/(1+jnp.exp(beta*(gcs_min - 12))) + 1/(1+jnp.exp(beta*(gcs_min - 14)))
  return cns_sofa_cont

def h_coagulation_discrete_original(platelet_min):
  if platelet_min < 20:
    coagulation_sofa = 4
  elif platelet_min < 50:
    coagulation_sofa = 3
  elif platelet_min < 100:
    coagulation_sofa = 2
  elif platelet_min < 150:
    coagulation_sofa = 1
  elif platelet_min is None:
    coagulation_sofa = None
  else:
    coagulation_sofa = 0
  return coagulation_sofa

def h_coagulation_cont(platelet_min, beta=15):
  coagulation_sofa_cont = 1/(1+jnp.exp(beta*((platelet_min/100)-0.2))) + 1/(1+jnp.exp(beta*((platelet_min/100)-0.5))) + 1/(1+jnp.exp(beta*((platelet_min/100)-1))) + 1/(1+jnp.exp(beta*((platelet_min/100)-1.5)))
  return coagulation_sofa_cont

def h_liver_discrete_original(bilirubin_max):
  if bilirubin_max is None:
    liver_sofa = None
  elif bilirubin_max >= 12.0:
    liver_sofa = 4
  elif bilirubin_max >= 6.0:
    liver_sofa = 3
  elif bilirubin_max >= 2.0:
    liver_sofa = 2
  elif bilirubin_max >= 1.2:
    liver_sofa = 1
  else:
    liver_sofa = 0
  return liver_sofa

def h_liver_cont(bilirubin_max , beta=2):
  liver_sofa_cont = 1/(1+jnp.exp(beta*(-(bilirubin_max - 12)))) + 1/(1+jnp.exp(beta*(-(bilirubin_max - 6)))) + 1/(1+jnp.exp(beta*(-(bilirubin_max - 2)))) + 1/(1+jnp.exp(beta*(-(bilirubin_max - 1.2))))
  return liver_sofa_cont

def h_sepsis_disc(patient_values):
  respiration = h_respiration_discrete_original(patient_values["respiration"][0][0],patient_values["respiration"][1][0])
  coagulation = h_coagulation_discrete_original(patient_values["coagulation"][0][0])
  liver = h_liver_discrete_original(patient_values["liver"][0][0])
  cardiovascular = h_cardiovascular_discrete_original(patient_values["cardiovascular"][0][0],patient_values["cardiovascular"][1][0], patient_values["cardiovascular"][2][0], patient_values["cardiovascular"][3][0], patient_values["cardiovascular"][4][0])
  cns = h_cns_discrete_original(patient_values["cns"][0][0])
  renal = h_renal_discrete_original(patient_values["renal"][0][0], patient_values["renal"][1][0])
  sofa_score = respiration + coagulation + liver + cardiovascular + cns + renal
  # return sofa_score
  if sofa_score >= 2:
    return 1
  return 0

def h_sofa_cont(treatment):
  respiration = h_respiration_cont(treatment[0],treatment[1])
  coagulation = h_coagulation_cont(treatment[2])
  liver = h_liver_cont(treatment[3])
  cardiovascular = h_cardiovascular_cont(treatment[4],treatment[5], treatment[6], treatment[7], treatment[8])
  cns = h_cns_cont(treatment[9])
  renal = h_renal_cont(treatment[10], treatment[11])
  sum_of_scores = respiration + coagulation + liver + cardiovascular + cns + renal
  return sum_of_scores

def h_sepsis_cont(treatment):
  sum_of_scores = h_sofa_cont(treatment)
  sepsis = 1/(1+jnp.exp(-(sum_of_scores - 2)))
  return sepsis

list_of_treatments = []
for patient in list_of_patient_values:
  patient_vals = []
  patient_vals.append(patient["respiration"][0][0])
  patient_vals.append(patient["respiration"][1][0])
  patient_vals.append(patient["coagulation"][0][0])
  patient_vals.append(patient["liver"][0][0])
  patient_vals.append(patient["cardiovascular"][0][0])
  patient_vals.append(patient["cardiovascular"][1][0])
  patient_vals.append(patient["cardiovascular"][2][0])
  patient_vals.append(patient["cardiovascular"][3][0])
  patient_vals.append(patient["cardiovascular"][4][0])
  patient_vals.append(patient["cns"][0][0])
  patient_vals.append(patient["renal"][0][0])
  patient_vals.append(patient["renal"][1][0])
  list_of_treatments.append(patient_vals)

for i,treatment_entry in enumerate(list_of_treatments[8050:]):
  try:
    confounder_val = h_sepsis_cont(treatment_entry)
    confounder_idx = 8050 + i
    print("Confounder Index", confounder_idx)
    print("Confounder Val Sepsis: ", confounder_val)
    break
  except Exception:
    continue

output_rows.append(list_of_rows[confounder_idx])

def flow_eqn_sepsis(t, confounder):
  flow = jnp.power((h_sepsis_cont(t) - confounder), 2)
  return flow

gradient_flow_eqn_sepsis = jax.grad(flow_eqn_sepsis, 0)


#Define parameters
h = 0.1 # Step size
s0_with_index_value = sample(list(enumerate(list_of_treatments)), 1)[0]
s0_index = s0_with_index_value[0]
print("Treatment Index", s0_index)
s0 = s0_with_index_value[1]
assert s0 != list_of_treatments[confounder_idx]

treatment_og = list_of_rows[s0_index]

output_rows.append(treatment_og)

# Explicit Euler Method
s = []
s.append(s0)

t2_t = {"t2":list_of_treatments[confounder_idx],"confounder_val_sepsis":float(confounder_val),"t*":s0}

for i in range(0, 500): 
  print(i)
  next = []
  gradients_to_subtract = []
  for j, gradient_entry in enumerate(gradient_flow_eqn_sepsis(s[i],confounder_val)):
    gradients_to_subtract.append(h*gradient_entry)
  print("Gradients: ", gradients_to_subtract)
  for k,_ in enumerate(s[i]):
    next.append(s[i][k] - gradients_to_subtract[k])
  s.append(next)

surrogate_treatment = np.asarray(s[-1]).tolist()


treatment_copy_1 = deepcopy(treatment_og)

treatment_copy_1["pao2fio2ratio_novent"] = surrogate_treatment[0]
treatment_copy_1["pao2fio2ratio_vent"] = surrogate_treatment[1]
treatment_copy_1["platelet_min"] = surrogate_treatment[2]
treatment_copy_1["bilirubin_max"] = surrogate_treatment[3]
treatment_copy_1["rate_dopamine"] = surrogate_treatment[4]
treatment_copy_1["rate_epinephrine"] = surrogate_treatment[5]
treatment_copy_1["rate_norepinephrine"] = surrogate_treatment[6]
treatment_copy_1["rate_dobutamine"] = surrogate_treatment[7]
treatment_copy_1["meanbp_min"] = surrogate_treatment[8]
treatment_copy_1["gcs_min"] = surrogate_treatment[9]
treatment_copy_1["creatinine_max"] = surrogate_treatment[10]
treatment_copy_1["uo_24hr"] = surrogate_treatment[11]

output_rows.append(treatment_copy_1)

t2_t["t'_sepsis"] = surrogate_treatment

confounder_val_sofa = h_sofa_cont(list_of_treatments[confounder_idx])
print("Confounder Val SOFA", confounder_val_sofa)

def flow_eqn_sofa(t, confounder):
  flow = jnp.power((h_sofa_cont(t) - confounder), 2)
  return flow

gradient_flow_eqn_sofa = jax.grad(flow_eqn_sofa, 0)

s0 = s[0]
assert s0 != list_of_treatments[confounder_idx]

s = []
s.append(s0)

for i in range(0, 200): 
  print(i)
  next = []
  gradients_to_subtract = []
  for j, gradient_entry in enumerate(gradient_flow_eqn_sofa(s[i],confounder_val_sofa)):
    gradients_to_subtract.append(h*gradient_entry)
  print("Gradients: ", gradients_to_subtract)
  for k,_ in enumerate(s[i]):
    next.append(s[i][k] - gradients_to_subtract[k])
  s.append(next)

surrogate_treatment = np.asarray(s[-1]).tolist()

treatment_copy_2 = deepcopy(treatment_og)

treatment_copy_2["pao2fio2ratio_novent"] = surrogate_treatment[0]
treatment_copy_2["pao2fio2ratio_vent"] = surrogate_treatment[1]
treatment_copy_2["platelet_min"] = surrogate_treatment[2]
treatment_copy_2["bilirubin_max"] = surrogate_treatment[3]
treatment_copy_2["rate_dopamine"] = surrogate_treatment[4]
treatment_copy_2["rate_epinephrine"] = surrogate_treatment[5]
treatment_copy_2["rate_norepinephrine"] = surrogate_treatment[6]
treatment_copy_2["rate_dobutamine"] = surrogate_treatment[7]
treatment_copy_2["meanbp_min"] = surrogate_treatment[8]
treatment_copy_2["gcs_min"] = surrogate_treatment[9]
treatment_copy_2["creatinine_max"] = surrogate_treatment[10]
treatment_copy_2["uo_24hr"] = surrogate_treatment[11]

output_rows.append(treatment_copy_2)

t2_t["confounder_sofa"] = float(confounder_val_sofa)

t2_t["t'_sofa"] = surrogate_treatment

with open("/content/attempt15_sepsis.json", "w+") as json_file:
  json.dump(t2_t, json_file)

column_names = output_rows.pop(0)
output_df = pd.DataFrame(output_rows, columns=column_names)
dropColList = ["rn_charttime",
              "rn_sofa_stay",
              "sofa_time", 
              "charttime", 
              "antibiotic_time", 
              "culture_time",
              "suspected_infection_time",
              "death_time",
              "subject_id",
              "stay_id",
              "admittime",
              "dischtime",
              "Unnamed: 0",
              "Unnamed: 0.1",
              "subject_id_1",
              "stay_id_1",
              "hadm_id",
              "sepsis3",
              "respiration",
              "coagulation",
              "cardiovascular",
              "renal",
              "cns",
              "liver",
              "mbp",
              "ICULOS",
              "los_hospital"]

for column_entry in dropColList:
  output_df = output_df.drop(column_entry, axis=1)

output_df.to_csv("pat4.csv")





In [ ]:
sepsis_real = []
sepsis_model = []
i = 0
while i < len(list_of_patient_values):
  real = h_sepsis_disc(list_of_patient_values[i])
  model = h_sofa_cont(list_of_treatments[i])
  sepsis_real.append(real)
  sepsis_model.append(model)
  i = i+100
  
plt.scatter(sepsis_real, sepsis_model)
plt.show()

In [ ]:
x = []
y_disc = []
y_cont =[]

for i in range(0, 25):
  x.append(i)
  y_disc.append(h_cardiovascular_discrete_original(i, i, i, i, i))
  y_cont.append(h_cardiovascular_cont(i, i, i, i, i))

fig1, ax = plt.subplots()
ax.plot(x, y_disc)
ax.plot(x, y_cont)
plt.show()
